In [ ]:
import importlib
import pandas as pd
import numpy as np
import re

from functions.functions_dataframe import * 
from parameters.filters_lists import *
from parameters.route import *
from parameters.regs import *

pd.set_option("display.max_colwidth", 200)

###### Step 1. Завантажити датафрейми ProZorro, МОН, створити список ЄДРПОУ освітніх закладів

In [ ]:
"""
Завантажити базу закупівель ProZorro (деталі у README.txt)
"""
df_prozorro = pd.read_csv('data/TrainingProZorroForSchoolProject20162019.csv', sep = ';', low_memory = False)

In [ ]:
"""
Завантажити датафрейм МОН навчальниз закладів, підготовлений у "fileter MON data.ipynb"
"""
df_mon_data_bez_upravlin_osvity = pd.read_csv('data/MON_schools_dnz.csv', sep = ';')

In [ ]:
"""
Створити список унікальних ЄДРПОУ освітніх закладів МОН
"""
mon_data_list = df_mon_data_bez_upravlin_osvity.edrpou.unique().tolist()

### ШКОЛИ ЗАМОВНИКИ

##### Step 2: Створити базу навчальних закладів, які самостійно купують через ProZorro

In [ ]:
"""
Відфільтрувати базу закупівель df_prozorro за колонкою IDOrganizator.
Фільтром є списко МОН з ЄДРПОУ начальних закладів mon_data_list (попередній крок)

df_tenders_by_schools - база закупівель, де навчальні заклади є замовниками
df_undefined_1 - база невизначених закупівель ProZorro
"""
df_tenders_by_schools = df_prozorro[df_prozorro['IDOrganizator'].isin(mon_data_list)]
df_undefined_1 = df_prozorro[~df_prozorro['IDOrganizator'].isin(mon_data_list)]

##### Step 3: Класифікація шкіл-замовників по типах за колонкою "Организатор" та нумерація

In [ ]:
df_tenders_by_schools_classified = classification_by_type_one_column(df_tenders_by_schools, ['Организатор'], filter_priority, filter_name_priority, 'Тип закладу')
df_tenders_by_schools_numerated = numeration_one_column(df_tenders_by_schools_classified, ['Организатор'], reg_number)

##### Step 4: Cтворення фінальної бази ШКІЛ ЗАМОВНИКІВ

In [ ]:
"""
"Назва закладу" - колонка з фіналізованою назвою школи. Принцип визначення:
    1. Якщо колонка "Номер" не є пустою - додається до "Типу закладу"
    2. Якщо колонка "Номер" є пустою - "Назва закладу" отримує значення "Организатор"
    
* колонка "Назва" не враховується через високий ступінь повторюваності з "Организатор" і часткову точність
"""
df_tenders_by_schools = df_tenders_by_schools_numerated.copy()
df_tenders_by_schools['Номер'] = df_tenders_by_schools['Номер'].replace(r'^\s*$', np.nan, regex=True) # технічна дія: перетворити пусті строки в NaN
df_tenders_by_schools['Назва закладу'] = np.where(df_tenders_by_schools['Номер'].notnull(), 
                                                  df_tenders_by_schools['Тип закладу'] + ' ' + df_tenders_by_schools['Номер'], 
                                                  df_tenders_by_schools['Организатор'])

In [ ]:
"""
Створення колонки 'Заклад-замовник'
Видалення "технічних" колонок, створених функціями.
"""
df_tenders_by_schools['Заклад-замовник'] = True
col_names = df_prozorro.columns.tolist()
col_names += ['Заклад-замовник', 'Назва закладу']
df_tenders_by_schools = df_tenders_by_schools[col_names]

### ШКОЛИ НЕ-ЗАМОВНИКИ

##### Step 5: Пошук закупівель шкіл-не-замовників за допомогою ключових слів

In [ ]:
"""
З-поміж датафрейму невизначених закупівель (df_undefined_1) за ключовими словами у списках filter_keywords, filter_keywords_strict 
відбираються закупівлі освітніх закладів у колонках: Тендер, ОписаниеТендера, Лот 

Регулярний вираз /b{}/b шукає за ключовими словами зі списку filter_keywords_strict_extended тільки 
ті паттерни, де ці ключові слова є окремими, а не зустрічають всередині слів. 
Це зменшить кількість "брудних" даних
"""
df_tenders_not_by_schools_by_keywords = df_undefined_1[df_undefined_1['Тендер'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords)), 
            case = False, na = False, regex = True)  | 
                                                      df_undefined_1['ОписаниеТендера'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords)), 
            case = False, na = False, regex = True) | 
                                                      df_undefined_1['Лот'].astype(str).str.contains(r'({})'.format('|'.join(filter_keywords)), 
            case = False, na = False, regex = True) | df_undefined_1['Тендер'].astype(str).str.contains(r'(/b{}/b)'.format('|'.join(filter_keywords_strict)), 
            case = False, na = False, regex = True) | df_undefined_1['ОписаниеТендера'].astype(str).str.contains(r'(/b{}/b)'.format('|'.join(filter_keywords_strict)), 
            case = False, na = False, regex = True) | df_undefined_1['Лот'].astype(str).str.contains(r'(/b{}/b)'.format('|'.join(filter_keywords_strict)), 
            case = False, na = False, regex = True)]

##### Step 6: Класифікація шкіл-не-замовників за типами

In [ ]:
df_tenders_not_by_schools_classified = classification_by_type_three_cols(df_tenders_not_by_schools_by_keywords,
                                                                         ['Тендер', 'ОписаниеТендера', 'Лот'], 
                                                                         filter_priority, 
                                                                         filter_name_priority)

##### Step 7: Нумерація шкіл-не-замовників

In [ ]:
"""
df_step7 - датафрейм, створений із закупівель df_tenders_not_by_schools_classified, де був успішно визначений "Тип закладу"
"""
df_step7 = df_tenders_not_by_schools_classified[df_tenders_not_by_schools_classified['Тип закладу'] != 'Невідомо'].copy()

In [ ]:
"""
Функція long_numeration створює два дафайреми:
    df_tenders_not_by_schools_numerated - тендери, де номер визначився 
    df_step7_numerated_undefined - тендери, де номер не визначився
"""
df_tenders_not_by_schools_numerated, df_step7_numerated_undefined = long_numeration(df_step7, reg_number)
df_tenders_not_by_schools_numerated['Назва закладу'] = df_tenders_not_by_schools_numerated['Тип закладу'] + ' ' + df_tenders_not_by_schools_numerated['Номер']

##### Step 8: Найменування шкіл-не-замовників

In [ ]:
"""
Завантажити датафрейм МОН власних назв навчальниз закладів, підготовлений у "fileter MON data.ipynb"
school_names - список унікальних назв 
"""
df_names_clean = pd.read_csv('data/MON_names.csv', sep = ';')
df_names_clean = df_names_clean[df_names_clean['Назва'].notnull()] # Видалити NaN
school_names = df_names_clean['Назва'].unique().tolist()

In [ ]:
"""
Функція long_naming створює два дафайреми:
    df_tenders_not_by_schools_named - тендери, де номер визначився.
    df_tenders_not_by_schools_undefined - тендери шкіл не замовників, де не визначилась назва (і номер).
    Такі закупівлі вважаються закупівлями управінь освіти для різних закладів.
"""
df_tenders_not_by_schools_named, df_tenders_not_by_schools_undefined = long_naming(df_step7_numerated_undefined, reg_double_quotes, school_names)
df_tenders_not_by_schools_named['Назва закладу'] = df_tenders_not_by_schools_named['Тип закладу'] + ' ' + df_tenders_not_by_schools_named['Назва']

##### Step 9: Об'єднання двох датафреймів шкіл-не-замовників

In [ ]:
"""
df_tenders_not_by_schools - датафрейм шкіл-не-замовників з визначеними номерами і назвами
Створення колонки 'Заклад-замовник'
"""
frames = [df_tenders_not_by_schools_numerated, df_tenders_not_by_schools_named]
df_tenders_not_by_schools = pd.concat(frames)
df_tenders_not_by_schools['Заклад-замовник'] = False
df_tenders_not_by_schools = df_tenders_not_by_schools[col_names]

In [ ]:
df_tenders_not_by_schools_undefined.sample(50)

##### Step 10. Підготовка датафрейму з координатами

In [ ]:
"""
df_coordinates - база даних шкіл МОН з 24 містами обласних центрів України, включно з координатами
"""
df_coordinates = pd.read_csv('data/coordinates/coordinates_DNZ _SC.csv', sep=';')

In [ ]:
"""
    B df_coordinates cтворити нову колонку ЄДРПОУ без помилкових текстових символів
"""
df_coordinates['edrpou_clean'] = df_coordinates.edrpou.str.extract('(\d+)', expand=False)
df_coordinates['edrpou_clean'] = df_coordinates['edrpou_clean'].fillna(0)
df_coordinates['edrpou_clean'] = pd.to_numeric(df_coordinates['edrpou_clean'])
df_coordinates['edrpou_clean'] = df_coordinates['edrpou_clean'].astype(np.int64)

In [ ]:
"""
    Заповнення відсутніх даних в базі МОН: заміна стрінги '(null)' на "ДНЗ"
"""
df_coordinates['updated_education_type_name'] = np.where(df_coordinates.education_type_name == '(null)',
                                              'ДНЗ', df_coordinates.education_type_name)

In [ ]:
"""
    Пошук і визначення типу закладу в колонках 'university_name' та 'updated_education_type_name'
"""
df_coordinates = classification_by_type_one_column(df_coordinates, ['university_name'], filter_priority, filter_name_priority, 'Тип закладу 1')
df_coordinates = classification_by_type_one_column(df_coordinates, ['updated_education_type_name'], filter_priority, filter_name_priority, 'Тип закладу 2')
df_coordinates['Тип закладу'] = np.where(~df_coordinates['Тип закладу 1'].str.contains('Невідомо'),
                                           df_coordinates['Тип закладу 1'],
                                           df_coordinates['Тип закладу 2'])
df_coordinates['Тип закладу'] = np.where(~df_coordinates['Тип закладу'].str.contains('Невідомо'),
                                           df_coordinates['Тип закладу'],
                                           df_coordinates['updated_education_type_name'])
df_coordinates['Тип закладу'] = df_coordinates['Тип закладу'].str.replace('загальносвітня школа', 'Загальносвітня школа')

In [ ]:
"""
    df_coordinates_numerated - виокремлення номерів шкільних закладів з бази МОН 
"""
df_coordinates_numerated = numeration_one_column(df_coordinates, ['university_name'], reg_number)

In [ ]:
"""
    df_coordinates_named - виокремлення власні назви шкільних закладів з бази МОН.
    Створення колонки "id" з визначених двома способами власної назви закладу
"""
df_coordinates_named = naming(df_coordinates, ['university_name'], reg_double_quotes, "naming")
df_coordinates_named['Назва'] = np.where(df_coordinates_named.university_name_naming_first.notnull(),
                                             df_coordinates_named.university_name_naming_first,
                                             df_coordinates_named.university_name_naming_last)
"""
    Створення колонки "Назва закладу". Там, де є номер - присвоєюється номер, в інших випадках - власна назва
"""
df_coordinates_named['Назва закладу'] = np.where(df_coordinates_named['Номер'].str.contains('№'), 
                                               df_coordinates['Тип закладу'] + ' '+ df_coordinates_named['Номер'],
                                               df_coordinates['Тип закладу'] + ' '+ df_coordinates_named['Назва'])
df_coordinates_named['id'] =  df_coordinates_named.koatuu_name_short + ' ' + df_coordinates_named['Назва закладу']

In [ ]:
"""    
    Створення датафрейму df_coordinates_with_id з id як назвою закладу та унікальним ідентифікатором
"""

cols_to_include = ['id', 'Longitude', 'Latitude', 'edrpou_clean']
df_coordinates_with_id = df_coordinates_named[cols_to_include]
df_coordinates_with_id['id'] = df_coordinates_with_id['id'].str.replace('№ ', '№')

##### Step 11: Об'єднання датафреймів шкіл-не-замовників та шкіл-замовників

In [ ]:
"""
df_tenders - Тендери з новими конолоками: "city", "id"
"""
frames = [df_tenders_by_schools, df_tenders_not_by_schools]
df_tenders = pd.concat(frames)
df_tenders = classification_by_type_one_column(df_tenders, ['Главный орган'], golovuj_organ_list, cities_list, 'city')
df_tenders['id'] = df_tenders['city'] + ' ' + df_tenders['Назва закладу']
df_tenders['id'] = df_tenders['id'].str.replace('№ ', '№')

##### Step 12: Створення готового датафрейму шкіл-замовників

In [ ]:
"""
df_tenders_true - тендери шкіл-замовників
"""
df_tenders_true = df_tenders[df_tenders['Заклад-замовник'] == True]
df_tenders_true.id = np.where(df_tenders_true.id.str.contains('Невідомо'), 
                              df_tenders_true.id.str.replace('Невідомо', filter_shkola_name), 
                              df_tenders_true.id)

In [ ]:
"""
tenders_by_schools - тендери шкіл-замовників з координатами і містом
"""
tenders_by_schools_completed = df_tenders_true.merge(df_coordinates_with_id, left_on='IDOrganizator', right_on='edrpou_clean')
tenders_by_schools_completed['id'] = np.where(tenders_by_schools_completed.id_x.str.contains('Невідомо'),
                                  tenders_by_schools_completed.id_y,
                                  tenders_by_schools_completed.id_x)

###### Step 13. Створення датафрейму шкіл-не-замовників

In [ ]:
"""
df_tenders_false - тендери шкіл-не-замовників
"""
df_tenders_false = df_tenders[df_tenders['Заклад-замовник'] == False]

In [ ]:
"""
    tenders_not_by_schools - фінальна таблиця шкіл-не-замовників
"""
tenders_not_by_schools = pd.merge(df_tenders_false, 
                                  df_coordinates_with_id, 
                                  on='id', 
                                  how='left')

In [ ]:
# tenders_not_by_schools[tenders_not_by_schools.Latitude.isnull()]
# len(tenders_not_by_schools[tenders_not_by_schools.Latitude.isnull()].id.unique().tolist())
# tenders_not_by_schools[tenders_not_by_schools.Latitude.isnull()].sample(3)

##### Step 14. Визначення координат Google Geolocator API у школах за назвою закладу в колонці 'id'

In [ ]:
tenders_without_coordinates = tenders_not_by_schools[tenders_not_by_schools.Latitude.isnull()]
tenders_without_coordinates.drop(['Longitude', 'Latitude'], axis=1, inplace = True)

In [ ]:
"""
    tenders_with_coordinates - датафрейм закупівель шкіл-не-замовників, 
    де не визначились координати раніше
"""

unique_school_names = tenders_without_coordinates.id.unique().tolist()
df_unique_school_names = pd.DataFrame(unique_school_names, columns =['id'])

# df_with_coordinates = get_coordinates(df_unique_school_names, 'id', 'Address')
# df_with_coordinates.to_excel("data/coordinates/df_with_coordinates.xlsx", index=False)
# df_with_coordinates.to_csv("data/coordinates/df_with_coordinates.csv", index=False, sep=';')

df_with_coordinates = pd.read_csv('data/coordinates/df_with_coordinates.csv', sep=';')

tenders_with_coordinates = pd.merge(tenders_without_coordinates, 
                                  df_with_coordinates, 
                                  on='id', 
                                  how='left')

##### Step 15: Створення готового датафрейму шкіл-не-замовників

In [ ]:
tenders_not_by_schools_completed = pd.concat([tenders_with_coordinates, tenders_not_by_schools[tenders_not_by_schools.Latitude.notnull()]])

##### Step 16: Створення готового датафрейму всіх навчальних закладів

In [ ]:
final_talbe_columns = ['id', 'city', 'Назва закладу', 'ИдентификаторЛота', 'Идентификатор', 'Организатор', 'Тендер',
       'СуммаЛота', 'ОписаниеТендера', 'Лот', 'Адрес поставки',
       'АдресОрганизатора', 'Главный орган', 'IDOrganizator',
       'Заклад-замовник', 'Longitude', 'Latitude']

tenders_by_schools_completed = tenders_by_schools_completed[final_talbe_columns]
tenders_not_by_schools_completed = tenders_not_by_schools_completed[final_talbe_columns]

final_df = pd.concat([tenders_by_schools_completed, tenders_not_by_schools_completed])